In [1]:
#imports:

#Numpy
import numpy as  np
#STL image creator
from stl import mesh
#Plotting
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
#Fits opening
from astropy.io import fits

#PSF
from astropy.convolution import Gaussian2DKernel,Box2DKernel
from astropy.convolution import convolve_fft

#resizing of images
import cv2

In [2]:
#Goal: First: to create a double model with a PSF (Gaussian) and a picture. Next: create a model where the result of 
#applying the PSF to the picture is shown.

In [25]:
#Function that creates triangles of the model that we want to print from the (x,y,z) positions array that we feed in.
#Can show the scaling radius as well if parameters are provided
def first_facemaker(Nparr,x_len,y_len,x_off,y_off,ax):
    ncols,nrows = Nparr.shape
    vertices=np.zeros((nrows,ncols,3))    
    height = Nparr.max()/3 +1.6
    for x in range(0, ncols):
        for y in range(0, nrows):
            z = Nparr[x][y]
            
            vertices[y][x]=(x/(ncols-1)*x_len+x_off, y/(nrows-1)*y_len+y_off, z)

    faces=[]
    
    #faces for the input array
    for x in range(0, ncols - 1):
        for y in range(0, nrows - 1):
            # create face 1
            vertice1 = vertices[y][x]
            vertice2 = vertices[y+1][x]
            vertice3 = vertices[y+1][x+1]
            face1 = np.array([vertice1,vertice2,vertice3])

            # create face 2 
            vertice1 = vertices[y][x]
            vertice2 = vertices[y][x+1]
            vertice3 = vertices[y+1][x+1]

            face2 = np.array([vertice1,vertice2,vertice3])

            faces.append(face1)
            faces.append(face2)
            
            

    #faces for the bottom layer and edges
    for i in range(0,4):
        for j in range(0,[ncols,nrows][i%2]-1):
            y1,x1 = [[[0,-1][i//2],j],[j,[0,-1][i//2]]][i%2]
            y2,x2 = [[[0,-1][i//2],j+1],[j+1,[0,-1][i//2]]][i%2]
            # create face 1 & 2
            vertice1 = vertices[y1][x1]
            vertice2 = vertices[y2][x2]
            vertice3 = (vertice1[0],vertice1[1],0)
            vertice4 = (vertice2[0],vertice2[1],0)

            face1 = np.array([vertice1,vertice3,vertice4])
            face2 = np.array([vertice1,vertice2,vertice4])

            faces.append(face1)
            faces.append(face2)
            
    faces.append(np.array([(x_off,y_off,0),(x_off,y_len+y_off,0),(x_len+x_off,y_len+y_off,0)]))
    faces.append(np.array([(x_off,y_off,0),(x_len+x_off,y_off,0),(x_len+x_off,y_len+y_off,0)]))
    
    #Faces for the border    
    offset = 3
    x1,y1 = [[0,int(ncols/2-offset)],[0,int(nrows/2-offset)]][ax-1]
    x2,y2 = [[0,int(ncols/2+offset)],[0,int(nrows/2+offset)]][ax-1]
    x3,y3 = [[nrows-1,int(ncols/2-offset)],[ncols-1,int(nrows/2-offset)]][ax-1]
    x4,y4 = [[nrows-1,int(ncols/2+offset)],[ncols-1,int(nrows/2+offset)]][ax-1]
    
    vertice1 = (vertices[x1][y1][0],vertices[x1][y1][1],0)
    vertice2 = (vertices[x2][y2][0],vertices[x2][y2][1],0)
    vertice3 = (vertices[x3][y3][0],vertices[x3][y3][1],0)
    vertice4 = (vertices[x4][y4][0],vertices[x4][y4][1],0)
    vertice5 = (vertice1[0],vertice1[1],height)
    vertice6 = (vertice2[0],vertice2[1],height)
    vertice7 = (vertice3[0],vertice3[1],height)
    vertice8 = (vertice4[0],vertice4[1],height)
              
    face1 = np.array([vertice1,vertice2,vertice6])
    face2 = np.array([vertice1,vertice5,vertice6])
    face3 = np.array([vertice3,vertice4,vertice8])
    face4 = np.array([vertice3,vertice7,vertice8])
    face5 = np.array([vertice1,vertice5,vertice7])
    face6 = np.array([vertice1,vertice3,vertice7])
    face7 = np.array([vertice2,vertice4,vertice8])
    face8 = np.array([vertice2,vertice6,vertice8])
    face9 = np.array([vertice1,vertice3,vertice4])
    face10 = np.array([vertice1,vertice2,vertice4])
    face11 = np.array([vertice5,vertice7,vertice8])
    face12 = np.array([vertice5,vertice6,vertice8])
              
    faces.append(face1)
    faces.append(face2)
    faces.append(face3)
    faces.append(face4)
    faces.append(face5)
    faces.append(face6)
    faces.append(face7)
    faces.append(face8)
    faces.append(face9)
    faces.append(face10)
    faces.append(face11)
    faces.append(face12)
              
    return faces

def second_facemaker(Nparr,x_len,y_len,x_off,y_off):
    ncols,nrows = Nparr.shape
    vertices=np.zeros((nrows,ncols,3))  
    for x in range(0, ncols):
        for y in range(0, nrows):
            z = Nparr[x][y]
            
            vertices[y][x]=(x/(ncols-1)*x_len+x_off, y/(nrows-1)*y_len+y_off, z)

    faces=[]
    
    #faces for the input array
    for x in range(0, ncols - 1):
        for y in range(0, nrows - 1):
            # create face 1
            vertice1 = vertices[y][x]
            vertice2 = vertices[y+1][x]
            vertice3 = vertices[y+1][x+1]
            face1 = np.array([vertice1,vertice2,vertice3])

            # create face 2 
            vertice1 = vertices[y][x]
            vertice2 = vertices[y][x+1]
            vertice3 = vertices[y+1][x+1]

            face2 = np.array([vertice1,vertice2,vertice3])

            faces.append(face1)
            faces.append(face2)
            
            

    #faces for the bottom layer and edges
    for i in range(0,4):
        for j in range(0,[ncols,nrows][i%2]-1):
            y1,x1 = [[[0,-1][i//2],j],[j,[0,-1][i//2]]][i%2]
            y2,x2 = [[[0,-1][i//2],j+1],[j+1,[0,-1][i//2]]][i%2]
            # create face 1 & 2
            vertice1 = vertices[y1][x1]
            vertice2 = vertices[y2][x2]
            vertice3 = (vertice1[0],vertice1[1],0)
            vertice4 = (vertice2[0],vertice2[1],0)

            face1 = np.array([vertice1,vertice3,vertice4])
            face2 = np.array([vertice1,vertice2,vertice4])

            faces.append(face1)
            faces.append(face2)
            
    faces.append(np.array([(x_off,y_off,0),(x_off,y_len+y_off,0),(x_len+x_off,y_len+y_off,0)]))
    faces.append(np.array([(x_off,y_off,0),(x_len+x_off,y_off,0),(x_len+x_off,y_len+y_off,0)]))             
    return faces

def Gaussian2D(x,y,x_off,y_off,sigma):
    return np.exp(-0.5*((x-x_off)**2+(y-y_off)**2)/sigma**2)

def Box2D(x,y,x_off,y_off,sigma):
    if abs(x-x_off) <= sigma and abs(y-y_off) <= sigma:
        z = 1
    else: z = 0
    return z

def realPSF2D(x,y,x_off,y_off,a):
    r = a*np.sqrt((x-x_off)**2+(y-y_off)**2)+0.01
    z = np.sin(r)/r
    return z

In [53]:
directory = 'images'


sigmas = [2,4,8]
max_sigma = max(sigmas)

for k,sigma in enumerate(sigmas):
    #F = fits.open(directory+'/YZ_BOO_R_1.fits') 

    #imageNp = originalimageNp[125:500,400:875]

    max_height = 18
    
    starcoords = [[25,60],[75,20]]
    
    #Two stars depending on the psf
    psfdimageNp = np.zeros((100,100))
    for i in range(len(psfdimageNp)):
        for j in range(len(psfdimageNp.T)):
            psfdimageNp[i][j] = (realPSF2D(i,j,starcoords[0][0],starcoords[0][1],sigma/7.5) + realPSF2D(i,j,starcoords[1][0],starcoords[1][1],sigma/7.5) )
    
    psfd_min_pix_2 = psfdimageNp.min() 
    psfdimageNp -= psfd_min_pix_2
    psfd_max_pix = psfdimageNp.max()
    scaledpsfdimageNp = (psfdimageNp * max_height) / psfd_max_pix + 1.6

    ncols,nrows = scaledimageNp.shape

    if ncols <= nrows:
        x_len = 40
        y_len = int(nrows/ncols*x_len)
    else:
        y_len = 40
        x_len = int(ncols/nrows*y_len)  

    psfNp = np.zeros((ncols,nrows))

    for x in range(0, ncols):
        for y in range(0, nrows):
            psfNp[x][y] = max_height*realPSF2D(x/ncols*8,y/nrows*8,4,4,sigma)+1.6

    compNp = np.zeros((ncols*2,nrows))
    for x in range(0,2*ncols):
        for y in range(0,nrows):
            if x < ncols:
                compNp[x][y] = psfNp[x][y]-psfNp.min() + 1.6
            else:
                compNp[x][y] = scaledpsfdimageNp[x-ncols][y]

    faces = first_facemaker(compNp,x_len*2,y_len,0,0,1)

    print(f"number of faces: {len(faces)}")
    facesNp = np.array(faces)

    # Create the mesh
    surface = mesh.Mesh(np.zeros(facesNp.shape[0], dtype=mesh.Mesh.dtype))
    for i in range(len(faces)):
        for j in range(3):
            surface.vectors[i][j] = facesNp[i][j]

    # Write the mesh to a file so we can print it
    surface.save('PSF_{}.stl'.format(k+1))

#Different psf shapes?
#For sure Gaussian psfs with NGC_4147
#Maybe Gaussian and different shape psf with M_60

number of faces: 40608
number of faces: 40608
number of faces: 40608


In [17]:
print(np.array([[1,2],[0,-5]]).min())

-5


In [26]:
print(ncols,nrows)

100 100
